In [267]:
import pandas as pd
import numpy as np
import tensorflow as tf
import keras
from sklearn.neural_network import MLPClassifier
from sklearn.decomposition import PCA
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from imblearn.over_sampling import SMOTE
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.utils import resample

In [259]:
df = pd.read_csv('train.csv')  
X = df.drop("labels", axis=1).values
y = df["labels"].values
y = np.where(y == 1, 1, 0) 
# Apply SMOTE to address data imbalance
smote = SMOTE(random_state=42)
X, y = smote.fit_resample(X, y)
# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=42)

In [262]:
from sklearn.linear_model import LogisticRegression
# Define the base neural network
base_estimator = MLPClassifier(hidden_layer_sizes=(100,), activation='relu', solver='adam', random_state=42)

# # Create an instance of the AdaBoostClassifier, using the base neural network as the base estimator
# classifier = AdaBoostClassifier(base_estimator=base_nn, n_estimators=10, algorithm='SAMME', random_state=42)
# classifier.fit(X_train, y_train)

In [268]:
# Define the number of weak classifiers (base estimators)
n_estimators = 10

# Create a list to store the weak classifiers
weak_classifiers = []
# Initialize the weights
weights = np.ones(len(X_train)) / len(X_train)

# Training phase: train weak classifiers using AdaBoost
for _ in range(n_estimators):
    # Resample the training data based on the weights
    X_train_resampled, y_train_resampled = resample(X_train, y_train, sample_weight=weights, random_state=42)
    
    # Train the MLPClassifier model on the resampled data
    base_estimator.fit(X_train_resampled, y_train_resampled)
    
    # Append the weak classifier to the list
    weak_classifiers.append(base_estimator)
    
    # Update the training weights
    y_pred = base_estimator.predict(X_train)
    incorrect_predictions = y_pred != y_train
    error = np.sum(weights[incorrect_predictions]) / np.sum(weights)
    alpha = np.log((1 - error) / error)
    weights *= np.exp(alpha * incorrect_predictions)
    weights /= np.sum(weights)

TypeError: resample() got an unexpected keyword argument 'sample_weight'

In [251]:
y_pred=classifier.predict(X_test)

In [252]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)

In [253]:
from sklearn.metrics import accuracy_score
print ("Accuracy : ", accuracy_score(y_test, y_pred))

Accuracy :  0.9121212121212121


In [254]:
from sklearn.metrics import f1_score
f1 = f1_score(y_test, y_pred)
# Print the F1 score
print("F1 Score:", f1)

F1 Score: 0.9128381279519107


In [255]:
test_data = pd.read_csv('test.csv')
test_feat=test_data.drop("id", axis=1).values
test_feat
pred=[]
pred = classifier.predict(test_feat)

In [256]:
# Create a DataFrame with the index numbers and predictions
results = pd.DataFrame({'Index': test_data.index, 'Prediction': pred})

# Save the results to a CSV file
results.to_csv('results.csv', index=False)



    

In [219]:
pred

array([0, 0, 0, ..., 1, 0, 1])

In [220]:
d=pd.read_csv('results.csv')
d

,Index,Prediction
0,0,0
1,1,0
2,2,0
3,3,0
4,4,0
...,...,...
2245,2245,0
2246,2246,0
2247,2247,1
2248,2248,0
